## Ibovespa Stocks Analysis

---

#### 1. Bibliotecas Utilizadas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### 2. Conjunto de Dados (B3 Stocks)

- O conjunto de dados ***b3_stocks*** apresenta informações sobre as cotação de diferentes ações entre os anos de **1994 a 2020 (Junho)**

In [2]:
b3_stocks = pd.read_csv('../data/b3_stocks_1994_2020.csv')

In [4]:
b3_stocks.tail()

,datetime,ticker,open,close,high,low,volume
1809942,2020-06-30,WLMM4,17.84,18.50,18.99,17.84,105747.00
1809943,2020-06-30,WSON33,34.27,35.59,36.18,33.61,4928729.00
1809944,2020-06-30,XBOV11,93.57,93.29,94.36,93.29,10334.50
1809945,2020-06-30,XPIE11,92.98,91.00,93.50,91.00,75828.82
1809946,2020-06-30,YDUQ3,33.67,33.65,34.59,33.18,88390681.00
